In [5]:
!pip install requests_aws4auth
!export host=https://search-smartsearch-bdhdkttjx3nshmf2flseozrqly.cn-north-1.es.amazonaws.com.cn/

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com


In [91]:
host="https://search-smartsearch-bdhdkttjx3nshmf2flseozrqly.cn-north-1.es.amazonaws.com.cn/"

In [186]:
import boto3
import json
import requests
import time
from collections import defaultdict
from requests_aws4auth import AWS4Auth
import os
import logging

KNN_ENDPOINT_NAME = 'huggingface-inference-eb'
GTM_ENDPOINT_NAME = 'pytorch-inference-llm-v2'  #根据输入参数决定用哪个模型
RECALL_CONTENT_LEN = 2000  #召回的内容，向LLM传入的长度

if not host:
    host =os.environ.get('host') 

# retrieve secret manager value by key using boto3                                             
sm_client = boto3.client('secretsmanager')
master_user = sm_client.get_secret_value(SecretId='opensearch-master-user')['SecretString']
data= json.loads(master_user)
username = data.get('username')
password = data.get('password')
num_output = 10

awsauth = (username, password)


source_includes = ["title","sentence","paragraph","sentence_id","paragraph_id"]
runtime= boto3.client('runtime.sagemaker')
fields = ["sentence"]
headers = { "Content-Type": "application/json" }


def get_results(r):
    print("output:",num_output)
    print("RECALL_CONTENT_LEN:",RECALL_CONTENT_LEN)
    res = []
    clean = []
    for i in range(len(r['hits']['hits'])):
        h = r['hits']['hits'][i]
        if h['_source']['paragraph'] not in clean:
          clean.append(h['_source']['paragraph'])
          res.append('<第'+str(i+1)+'条信息>'+h['_source']['paragraph'] + '。</第'+str(i+1)+'条信息>\n')
    #print("length:",len(' '.join(res)))
    #print(' '.join(res))
    return ' '.join(res)

def get_vector(q):
  payload = json.dumps({"inputs":[q]}).encode('utf-8')
  response = runtime.invoke_endpoint(EndpointName=KNN_ENDPOINT_NAME,
                                     ContentType="application/json",
                                     Body=payload)
  json_res=json.loads(response['Body'].read().decode())
  if "sentence_embeddings" in json_res:                                   
    result = json_res["sentence_embeddings"][0][0]
  else:
    result = json_res[0][0][0]
    
  #result = json.loads(response['Body'].read().decode())[0][0][0]
  # print(result)
  return result
  
def get_answer(question, context):
  def query_endpoint_with_json_payload(encoded_json):
    response = runtime.invoke_endpoint(EndpointName=GTM_ENDPOINT_NAME, ContentType='application/json', Body=encoded_json)
    return response

  def parse_response_texts(query_response):
      model_predictions = json.loads(query_response['Body'].read())
      generated_text = model_predictions["answer"]
      return generated_text
  
  prompt_template = "已知内容:"
  
  prompt_template += context + """
  
基于以上已知信息，简洁和专业的来回答用户的问题，并告知是依据哪些信息来进行回答的。
答案请使用中文。
问题是:"""
     

  prompt_template += question
  payload = {"ask": prompt_template,"temperature":0.8}
  #print("*****************************************************************")
  print(prompt_template)

  query_response = query_endpoint_with_json_payload(json.dumps(payload).encode('utf-8'))
  generated_texts = parse_response_texts(query_response)
  return generated_texts
  
def search_using_knn(q, index, source_includes, size, k):
    #print(1, q)
    query = {
          "size": num_output,
          "_source": {
            "includes": source_includes
          },
          "size": size,
          "query": {
            "knn": {
              "sentence_vector": {
                "vector": get_vector(q),
                "k": k
              }
            }
          }
        }
    r = requests.post(host + index + '/_search', auth=awsauth, headers=headers, json=query)
    return r.text

def search_using_match(event, index, source_includes, size):
    q = event['queryStringParameters']['q']
      # q = split_sentences(q)
    if 'ml' not in event['queryStringParameters']:
      query = {
        "size": num_output,
          "_source": {
              "includes": source_includes
            },
        'query': {
          "multi_match": {
            "query": event['queryStringParameters']['q'],
            "fields": fields
          }
        }
      }
    else:
      query = {
        "size": num_output,
          "_source": {
              "includes": source_includes
            },
        "query": {
          "multi_match": {
            "query": event['queryStringParameters']['q'],
            "fields": fields
          }
        },
        "rescore": {
          "query": {
            "rescore_query": {
              "sltr": {
                "params": {
                  "keywords": event['queryStringParameters']['q']
                },
                "model": event['queryStringParameters']['ml']
              }
            }
          }
        }
      }
    r = requests.post(host + index + '/_search', auth=awsauth, headers=headers, json=query)
    return r.text

def split_sentences(q):
    def is_letter_or_number(s):
        if "0" <= s <= "9" or "a" <= s <= "z" or "A" <= s <= "Z":
            return True
        return False
    
    slices = []
    i = 0
    while i < len(q):
        if is_letter_or_number(q[i]):
            tmp = [q[i]]
            i += 1
            while i < len(q) and is_letter_or_number(q[i]):
                tmp .append(q[i])
                i += 1
            slices.append('*'+'*'.join(tmp)+'*')
            slices.append(''.join(tmp))
        else:
            tmp = q[i]
            i += 1
            while i < len(q) and is_letter_or_number(q[i])==False:
                tmp += q[i]
                i += 1
            slices.append(tmp)
    return ' '.join(slices)

# Lambda execution starts here
def lambda_handler(event, context):
    global RECALL_CONTENT_LEN
    global GTM_ENDPOINT_NAME
    global num_output
  
    index = event['queryStringParameters']['ind']
    llm_version=event['queryStringParameters']['version']
    if llm_version=="1":
      GTM_ENDPOINT_NAME = 'pytorch-inference-llm-v1'
      num_output=10
      RECALL_CONTENT_LEN=2000
    else:
      GTM_ENDPOINT_NAME = 'pytorch-inference-llm-v2'
      num_output=6
      RECALL_CONTENT_LEN=1000
    print('index', index)
    if event['queryStringParameters']['knn'] == "1":
      print ("query:",event['queryStringParameters']['q'])
      r = search_using_knn(event['queryStringParameters']['q'], event['queryStringParameters']['ind'], 
                          source_includes, num_output,10)
      r_match=search_using_match(event,event['queryStringParameters']['ind'],source_includes, num_output)
      r_match_json=json.loads(r_match)
      print(r_match)
      if r_match_json and len(r_match_json['hits']['hits'])==0:
        print("通过分词搜索，更正向量化结果:")
        r=r_match
    else:
      r=search_using_match(event,event['queryStringParameters']['ind'],source_includes, num_output)
    
    #print(r)
    content = get_results(json.loads(r))
    #print(content[:RECALL_CONTENT_LEN])
    
    suggestion_answer = get_answer(event['queryStringParameters']['q'], content[:RECALL_CONTENT_LEN])
    

    display(Markdown(suggestion_answer))
    # # Create the response and add some extra content to support CORS
    response = {
        "statusCode": 200,
        "headers": {
            "Access-Control-Allow-Origin": '*'
        },
        "isBase64Encoded": False
    }
    response['body'] = json.dumps(
      {
        'body': json.loads(r), 
        'datetime':time.time(),
        'suggestion_answer': suggestion_answer
      })
    return response


In [188]:
%%time
event={
  "queryStringParameters": {
    "q": "心脏病发病风险与哪些因素有关？？",
    "knn": "1",
    "ind": "test",
    "version": "2"
  }
}
out=lambda_handler(event,"")


index test
query: 心脏病发病风险与哪些因素有关？？
{"took":8,"timed_out":false,"_shards":{"total":5,"successful":5,"skipped":0,"failed":0},"hits":{"total":{"value":1306,"relation":"eq"},"max_score":25.183184,"hits":[{"_index":"test","_type":"_doc","_id":"188","_score":25.183184,"_source":{"sentence":"\n心脏病发病风险均与身体活动不足有关。","paragraph":"因此，\n提出了“职业性的身体活动对预防冠心病有保护\n作用”这一假设。\n心脏病发病风险均与身体活动不足有关。 Bijnen\n对老年男性开展了为期 10年的随访，在调整年龄、\n吸烟等变量后，以每天身体活动 15 min者为基线，\n身体活动增加至 1 h/d者和 3 h/d者，心血管病死亡\n相对危险度分别下降至 0.75和0.70。另一项对中老\n年女性的 10年随访研究显示，在调整相关变量后，\n与轻度身体活动者相比，中度身体活动者脑卒中相\n对危险度为 0.77。","title":"基层心血管病综合管理实践指南2020"}},{"_index":"test","_type":"_doc","_id":"301","_score":22.07922,"_source":{"sentence":"\n1.9.2 高血压与心血管病风险 血压水平与心脑血\n管病发病和死亡风险存在密切的因果关系。","paragraph":"\n除了以上危险因素外，高血压其他危险因素还\n包括年龄、高血压家族史、缺乏体力活动，以及糖\n尿病、血脂异常等。近年来大气污染与血压的关联\n性也受到关注。\n1.9.2 高血压与心血管病风险 血压水平与心脑血\n管病发病和死亡风险存在密切的因果关系。 SBP每\n升高20 mmHg或DBP每升高10 mmHg，心脑血管病\n发生风险倍增。\n血压水平与心衰的发生也存在因果关系。高\n血压主要导致射血分数保留的心衰（ heart failure\nwith preserved ejection fractio

CPU times: user 46.8 ms, sys: 0 ns, total: 46.8 ms
Wall time: 372 ms


In [159]:
# 使用 Markdown 格式打印模型输出
from IPython.display import display, Markdown, clear_output

def display_answer(response_all):
    for response, history in response_all:
        clear_output(wait=True)
        display(Markdown(response))
    return history


In [160]:
display(Markdown(out))

TypeError: Markdown expects text, not {'statusCode': 200, 'headers': {'Access-Control-Allow-Origin': '*'}, 'isBase64Encoded': False, 'body': '{"body": {"took": 7, "timed_out": false, "_shards": {"total": 5, "successful": 5, "skipped": 0, "failed": 0}, "hits": {"total": {"value": 50, "relation": "eq"}, "max_score": 0.0065720603, "hits": [{"_index": "test", "_type": "_doc", "_id": "323", "_score": 0.0065720603, "_source": {"sentence": "\\u9ad8\\u8840\\u538b\\u662f\\u6211\\n\\u56fd\\u5fc3\\u8840\\u7ba1\\u75c5\\u7684\\u4e3b\\u8981\\u5371\\u9669\\u56e0\\u7d20\\uff0c\\u63a7\\u5236\\u9ad8\\u8840\\u538b\\u7684\\u76f8\\u5173\\u5f71\\n\\u54cd\\u56e0\\u7d20\\u662f\\u9632\\u6cbb\\u9ad8\\u8840\\u538b\\u4e43\\u81f3\\u5fc3\\u8840\\u7ba1\\u75c5\\u7684\\u91cd\\u8981\\u6240\\u5728\\u3002", "paragraph": "\\n\\u4e0e\\u5355\\u7eaf\\u7cd6\\u5c3f\\u75c5\\u60a3\\u8005\\u76f8\\u6bd4\\uff0c\\u540c\\u65f6\\u4f34\\u6709\\u9ad8\\u8840\\u538b\\u548c /\\n\\u6216\\u8840\\u8102\\u5f02\\u5e38\\u7684\\u7cd6\\u5c3f\\u75c